# Кольца Артем М.19-04

## Машинное обучение (2 семестр)

### Spark
### Подсчет количества слов в книге

### Задание №1

In [1]:
# Загрузка Spark Context
from pyspark import SparkContext
from pyspark.sql import SparkSession
import re, string

sc = SparkContext("local","PySpark Word Count")
text_file = sc.textFile('451_fahrenheit.txt')

In [2]:
# первые 5 элементов
text_file.take(5)

['451 градус по Фаренгейту',
 'Рэй Дуглас Брэдбери',
 '',
 'Часть первая. Домашний очаг и саламандра',
 '']

In [3]:
# пунктуация
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [5]:
def clean_str(x):
    lower_str = x.lower()
    lowercased = lower_str.replace('--',' ')
    clean_str = lowercased.translate(punc) # чистка от пунктуации
    return clean_str

In [7]:
book = text_file.map(lambda x: clean_str(x))
book.take(5)

['451 градус по фаренгейту',
 'рэй дуглас брэдбери',
 '',
 'часть первая. домашний очаг и саламандра',
 '']

In [8]:
book = text_file.map(lambda x: clean_str(x).split())
# разделение элементов в массивы
book.take(5)

[['451', 'градус', 'по', 'фаренгейту'],
 ['рэй', 'дуглас', 'брэдбери'],
 [],
 ['часть', 'первая.', 'домашний', 'очаг', 'и', 'саламандра'],
 []]

In [9]:
book_clean = text_file.flatMap(lambda x: clean_str(x).split())
# убраны пустые массивы
book_clean.take(5)

['451', 'градус', 'по', 'фаренгейту', 'рэй']

In [10]:
book_clean = text_file.flatMap(lambda x: clean_str(x).split())
book_word_counts = book_clean.map(lambda x: (x,1))
# подсчет каждого слова в книге
book_word_counts.take(5)

[('451', 1), ('градус', 1), ('по', 1), ('фаренгейту', 1), ('рэй', 1)]

In [11]:
book_clean = text_file.flatMap(lambda x: clean_str(x).split())
book_word_counts = book_clean.map(lambda x: (x,1))
book_word_counts = book_word_counts.reduceByKey(lambda x,y: x + y)
book_word_counts.take(15)

[('451', 2),
 ('градус', 1),
 ('по', 59),
 ('фаренгейту', 1),
 ('рэй', 1),
 ('дуглас', 1),
 ('брэдбери', 1),
 ('часть', 3),
 ('первая.', 1),
 ('домашний', 1),
 ('очаг', 1),
 ('и', 372),
 ('саламандра', 1),
 ('жечь', 1),
 ('было', 46)]

In [12]:
book_word_counts = book_word_counts.map(lambda x:(x[1],x[0]))
# первые 15 слов с сортировкой по убыванию
book_word_counts.sortByKey(False).take(15)

[(610, '–'),
 (372, 'и'),
 (323, 'в'),
 (246, 'не'),
 (190, 'на'),
 (150, 'он'),
 (124, 'что'),
 (100, 'с'),
 (99, 'я'),
 (91, 'она'),
 (90, 'а'),
 (89, 'как'),
 (78, 'это'),
 (77, 'его'),
 (70, 'монтаг')]